Keras Tuner es una biblioteca que nos ayuda a elegir el conjunto óptimo de hiperparámetros para nuestro programa TensorFlow. El proceso de seleccionar el conjunto correcto de hiperparámetros para su aplicación de aprendizaje automático (ML) se denomina ajuste de hiperparámetros o hypertuning.

Los hiperparámetros son las variables que gobiernan el proceso de entrenamiento y la topología de un modelo de ML. Estas variables permanecen constantes durante el proceso de capacitación e impactan directamente en el rendimiento del programa ML. Los hiperparámetros son de dos tipos:

1. Hiperparámetros del modelo que influyen en la selección del modelo, como el número y el ancho de las capas ocultas.

2. Hiperparámetros del algoritmo que influyen en la velocidad y la calidad del algoritmo de aprendizaje, como la tasa de aprendizaje para el descenso de gradiente estocástico (SGD) y el número de vecinos más cercanos para un clasificador de vecinos más cercanos (KNN)

Usaremos Keras Tuner para realizar un hiperajuste para una aplicación de clasificación de imágenes.

In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

Vamos a usar Keras Tuner para encontrar los mejores hiperparámetros para un modelo de ML que clasifica imágenes de ropa del conjunto de datos Fashion MNIST.

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [3]:
# Normalizamos los pixeles entre 0 y 1.
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Modelo.

Creamos un modelo para hiperajueste, también definimos el espacio de búsqueda de hiperparámetros además de la arquitectura del modelo. El modelo que configuramos para el hiperajuste se denomina hipermodelo.

Podemos definir un hipermodelo a través de dos enfoques:

* Mediante el uso de una función de generador de modelos.

* HyperModel. La clase HyperModel de la API Keras Tuner.

También podemos usar dos clases de HyperModel predefinidas: HyperXception e HyperResNet para aplicaciones de visión artificial.

Usaremos una función generador de modelos para definir el modelo de clasificación de imágenes. La función del generador de modelos devuelve un modelo compilado y usa hiperparámetros que usted define en línea para hiperafinar el modelo.

In [10]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape = (28, 28)))

    # Ajustamos el número de unidades en la primera capa densa.
    # Elegimos un valor óptimo entre 32 - 512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))

    # Ajustamos la tasa de aprendizaje para el optimizador.
    # Elegimos un valor óptimo entre 0,01, 0,001 o 0,0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                  metrics = ['accuracy'])
    
    return model

### Creamos una instancia de sintonizador y realizamos un hiperajuste.

Creemos una instancia del sintonizador para realizar el hiperajuste. Keras Tuner tiene cuatro sintonizadores disponibles: RandomSearch, Hyperband, BayesianOptimizer y Sklearn. Vamos a utilizar Hyperband.

Para instanciar el sintonizador Hyperband, debemos especificar el hipermodelo, el objetive a optimizar y el número máximo de épocas para entrenar(max_epochs).

In [11]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

El algoritmo de sintonización Hyperband utiliza la asignación de recursos adaptable y la detención anticipada para converger rápidamente en un modelo de alto rendimiento. Esto se hace usando un soporte estilo campeonato deportivo. El algoritmo entrena una gran cantidad de modelos durante algunas épocas y lleva adelante solo la mitad de los modelos con el mejor rendimientos a la siguiente ronda. Hyperband determina la cantidad de modelos para entrenar en un grupo calculando 1 + factor logarítmico(max_epochs) y redondeándolo al entero más cercano.



Creemos una callback para parar el entrenamiento antes de alcanzar cierto valor para la pérdida de validación.

In [12]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)

Ejecutamos la búsqueda de hiperparametros. Los argumentos para el método de búsqueda son los mismos que los utilizados para tf.keras.model.fit además de la callback anterior.

In [13]:
tuner.search(img_train, label_train, epochs = 50, validation_split = 0.2, callbacks = [stop_early])

# Obtenemos el hiperparámetro óptimo.
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
    La busqueda de hiperparámetros está completa. El número óptimo de unidades en la primera zona densamente
    conectada. La capa {best_hps.get('units')} y la tasa de aprendizaje óptima para el optmiziador es 
    {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 15s]
val_accuracy: 0.8755000233650208

Best val_accuracy So Far: 0.8949999809265137
Total elapsed time: 00h 20m 19s

    La busqueda de hiperparámetros está completa. El número óptimo de unidades en la primera zona densamente
    conectada. La capa 320 y la tasa de aprendizaje óptima para el optmiziador es 
    0.001.



### Entrenamos el modelo.

Encontremos el número óptimo de épocas para entrenar el modelo con los hiperparámetros obtenidos de la búsqueda.

In [14]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs = 50, validation_split = 0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Mejor época: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 10s 6ms/step - loss: 0.4954 - accuracy: 0.8235 - val_loss: 0.4440 - val_accuracy: 0.8340
Epoch 2/50
1500/1500 [==============================] - 11s 7ms/step - loss: 0.3737 - accuracy: 0.8649 - val_loss: 0.4075 - val_accuracy: 0.8551
Epoch 3/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3317 - accuracy: 0.8779 - val_loss: 0.3354 - val_accuracy: 0.8786
Epoch 4/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3058 - accuracy: 0.8854 - val_loss: 0.3577 - val_accuracy: 0.8718
Epoch 5/50
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2876 - accuracy: 0.8934 - val_loss: 0.3433 - val_accuracy: 0.8772
Epoch 6/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2725 - accuracy: 0.8980 - val_loss: 0.3205 - val_accuracy: 0.8835
Epoch 7/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2577 - accuracy: 0.9043 - val_loss: 0.3304 - val_accurac

Volvamos a crear instancias del hipermodelo y entrenemos con el número óptimo de épocas.

In [15]:
hypermodel = tuner.hypermodel.build(best_hps)

hypermodel.fit(img_train, label_train, epochs = best_epoch, validation_split = 0.2)

Epoch 1/42
1500/1500 [==============================] - 8s 4ms/step - loss: 0.4958 - accuracy: 0.8246 - val_loss: 0.3931 - val_accuracy: 0.8636
Epoch 2/42
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3719 - accuracy: 0.8649 - val_loss: 0.3933 - val_accuracy: 0.8580
Epoch 3/42
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3337 - accuracy: 0.8770 - val_loss: 0.3466 - val_accuracy: 0.8770
Epoch 4/42
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3075 - accuracy: 0.8865 - val_loss: 0.3273 - val_accuracy: 0.8865
Epoch 5/42
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2873 - accuracy: 0.8924 - val_loss: 0.3455 - val_accuracy: 0.8787
Epoch 6/42
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2715 - accuracy: 0.8979 - val_loss: 0.3316 - val_accuracy: 0.8827
Epoch 7/42
1500/1500 [==============================] - 7s 4ms/step - loss: 0.2588 - accuracy: 0.9036 - val_loss: 0.3482 - val_accuracy:

Evaluemos el hipermodelo con los datos de prueba.

In [16]:
eval_result = hypermodel.evaluate(img_test, label_test)
print('[test loss, test accuracy]:', eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.5171 - accuracy: 0.8865
[test loss, test accuracy]: [0.5170891880989075, 0.8865000009536743]


Y ya aprendimos a usar Keras Tuner para justar los hiperparámetros de un modelo.